Goal is to process cloud mask data from himawari satelite data into a single time series of cloud cover from 2019 to 2024 for Sydney Airport location, which is (-33.9354482,151.1618103)
the data can be downloaded using THREDDS Server, with a sample link
https://thredds.nci.org.au/thredds/fileServer/rv74/satellite-products/arc/der/himawari-ahi/cloud/cma/latest/2019/01/01/S_NWC_CMA_HIMA08_HIMA-N-NR_20190101T000000Z.nc
the data value is either 1, 0, or 255. 1 means cloudy, 0 means not cloudy, 255 means invalid / missing\

Geospatial string from the dataset, checked on 2 files, assumed to be the same for all files. 
proj4 = "+proj=geos +lon_0=140.7 +h=35785863 +x_0=0 +y_0=0 +a=6378137 +b=6356752.3 +units=m +no_defs";


In [2]:
import requests
import netCDF4
import io

# Data Exploration

In [ ]:
# Step 1: Define the URL of the .nc file
url = 'https://thredds.nci.org.au/thredds/fileServer/rv74/satellite-products/arc/der/himawari-ahi/cloud/cma/latest/2019/01/01/S_NWC_CMA_HIMA08_HIMA-N-NR_20190101T000000Z.nc'

# Step 2: Download the file from the URL into memory using `requests`
response = requests.get(url)

# Check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    # Step 3: Open the .nc file from the downloaded content using `netCDF4`
    dataset = netCDF4.Dataset('in-memory.nc', 'r', memory=response.content)

    # Step 4: Filter out only the variables 'cma', 'nx', and 'ny'
    cma_data = dataset.variables['cma'][:]
    nx_data = dataset.variables['nx'][:]
    ny_data = dataset.variables['ny'][:]

    # Step 5: Print the filtered data (optional for verification)
    print("cma data shape:", cma_data.shape)
    print("nx data shape:", nx_data.shape)
    print("ny data shape:", ny_data.shape)

    # Optionally: You can store or process the data as needed
    # For example, saving the data to a CSV file:
    # Convert the data to a pandas DataFrame and save to CSV (if needed)
    # import pandas as pd
    # df = pd.DataFrame({
    #     'nx': nx_data.flatten(),
    #     'ny': ny_data.flatten(),
    #     'cma': cma_data.flatten()
    # })
    
    # Save the DataFrame to CSV
    # df.to_csv('cloud_mask_data.csv', index=False)

    print("Data has been successfully processed and saved as cloud_mask_data.csv.")

else:
    print(f"Failed to download the file. Status code: {response.status_code}")



In [26]:
cma_data

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=255,
  dtype=uint8)

In [42]:
type(cma_data.data)

numpy.ndarray

In [44]:
import numpy as np

In [46]:
np.unique(cma_data.data, return_counts=True)

(array([  0,   1, 255], dtype=uint8), array([ 5354917, 17750827,  7144256]))

In [47]:
np.unique(cma_data.mask, return_counts=True)

(array([False,  True]), array([23105744,  7144256]))

In [48]:
nx_data

masked_array(data=[-5498995.5, -5496995.5, -5494995.5, ...,  5494995. ,
                    5496995. ,  5498995. ],
             mask=False,
       fill_value=np.float64(1e+20),
            dtype=float32)

# Finding the projected nx and ny of sydney airport

In [68]:
import pyproj

# Define the projection based on the provided proj4 string
proj = pyproj.Proj("+proj=geos +lon_0=140.7 +h=35785863 +a=6378137 +b=6356752.3 +units=m +no_defs")
# this is from the himawari proj4 string, which is used to convert latitude and longitude to projection coordinates

# Sydney airport latitude and longitude (in degrees)
lat = -33.9354482   # Latitude
lon = 151.1618103  # Longitude

# Convert the latitude/longitude to projection coordinates (nx, ny)
nx, ny = proj(lon, lat)

print(f"Projected coordinates: nx = {nx}, ny = {ny}")


Projected coordinates: nx = 931240.9112048375, ny = -3416933.5280751055


In [73]:
# Find the index of the closest value
closest_index = np.abs(nx_data.data - nx).argmin()

# Get the closest value using the index
closest_value = nx_data.data[closest_index]

print(f"The closest value to {nx} is {closest_value} which index is {closest_index}")


The closest value to 931240.9112048375 is 930999.0 which index is 3215


In [74]:
# Find the index of the closest value
closest_index = np.abs(ny_data.data - ny).argmin()

# Get the closest value using the index
closest_value = ny_data.data[closest_index]

print(f"The closest value to {ny} is {closest_value} which index is {closest_index}")


The closest value to -3416933.5280751055 is -3416997.0 which index is 4458


So, we're interested in nx = 930999.0 and ny = -3416997.0, which index is 3215 & 4458

In [76]:
nx_data[3215]

np.float32(930999.0)

In [75]:
ny_data[4458]

np.float32(-3416997.0)

In [86]:
cma_data.data[3215, 4458]

np.uint8(1)

In [82]:
cma_data.data[4458, 3215]
# This is important not to be wrong: we want [ny, nx] not [nx, ny] because y corresponds to the latitude (north-south) and 
# x corresponds to the longitude (east-west) in the projection system., and on the dataset it's specified that the data is shaped [ny, nx]

np.uint8(0)

In [79]:
import numpy as np

# Example 2D array with shape (3, 4)
array = np.array([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12]])

print("Shape of the array:", array.shape)

Shape of the array: (3, 4)


In [80]:
array[1,2]

np.int64(7)

# Obtain Cloud Data For Specific Time Step

In [87]:
# Step 1: Define the URL of the .nc file
url = 'https://thredds.nci.org.au/thredds/fileServer/rv74/satellite-products/arc/der/himawari-ahi/cloud/cma/latest/2019/01/01/S_NWC_CMA_HIMA08_HIMA-N-NR_20190101T000000Z.nc'

# Step 2: Download the file from the URL into memory using `requests`
response = requests.get(url)

# Check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    dataset = netCDF4.Dataset('in-memory.nc', 'r', memory=response.content)
    cloud_cover = int(dataset.variables['cma'][:].data[4458, 3215])
    # this is cloud cover for Sydney Airport location
    
else:
    print(f"Failed to download the file. Status code: {response.status_code}")



In [88]:
cloud_cover

0

# Try to See if BOM data from Darth has this data already

In [3]:
import pandas as pd

In [4]:
bom_data = pd.read_csv('../../../../data/1. raw/BOM Weather Data/bom_data_nsw55167608.csv', nrows = 1)

In [5]:
bom_data.columns

Index(['hm', 'station_number', 'state', 'year', 'month', 'day',
       'hour_local_std', 'min_local_std', 'datetime',
       'precipitation_since_9am_local_time_in_mm',
       'quality_of_precipitation_since_9am_local_time',
       'air_temperature_in_degrees_c', 'quality_of_air_temperature',
       'wet_bulb_temperature_in_degrees_c', 'quality_of_wet_bulb_temperature',
       'dew_point_temperature_in_degrees_c',
       'quality_of_dew_point_temperature', 'relative_humidity_in_percentage',
       'quality_of_relative_humidity', 'vapour_pressure_in_hpa',
       'quality_of_vapour_pressure', 'saturated_vapour_pressure_in_hpa',
       'quality_of_saturated_vapour_pressure', 'wind_speed_in_km_h',
       'wind_speed_quality', 'wind_direction_in_degrees_true',
       'wind_direction_quality',
       'speed_of_maximum_windgust_in_last_10_minutes_in_km_h',
       'quality_of_speed_of_maximum_windgust_in_last_10_minutes',
       'cloud_amount_of_first_group_in_eighths',
       'quality_of_firs

In [6]:
import pandas as pd

# Define the condition for filtering (stations you're interested in)
column_name = 'station_number'  # Replace with your column name
station_number = 66037

# Define the columns to read (make sure to include 'station_number' in col_to_include)
col_to_include = ['station_number', 'datetime', 
                    'cloud_amount_of_first_group_in_eighths',
                    'quality_of_first_group_of_cloud_amount',
                    'cloud_amountof_second_group_in_eighths',
                    'quality_of_second_group_of_cloud_amount',
                    'cloud_amount_of_third_group_in_eighths',
                    'quality_of_third_group_of_cloud_amount',
                    'cloud_amount_of_fourth_group_in_eighths',
                    'quality_of_fourth_group_of_cloud_amount',
                    'ceilometer_cloud_amount_of_first_group',
                    'quality_of_first_group_of_ceilometer_cloud_amount',
                    'ceilometer_cloud_amount_of_second_group',
                    'quality_of_second_group_of_ceilometer_cloud_amount',
                    'ceilometer_cloud_amount_of_third_group',
                    'quality_of_third_group_of_ceilometer_cloud_amount',
                    'ceilometer_sky_clear_flag']

# Specify the chunk size (e.g., 10000 rows per chunk)
chunk_size = 10000

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Iterate over the file in chunks
for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv', 
                         usecols=col_to_include, chunksize=chunk_size):
    # Filter rows where 'station_number' is in the list of value_conditions
    filtered_chunk = chunk[chunk[column_name] == station_number]
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks)

# Display the first 5 filtered rows
print(filtered_df.head(5))


C:\Users\z5404477\AppData\Local\Temp\ipykernel_4132\693871328.py:32: DtypeWarning: Columns (53,54,57,58,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv',
C:\Users\z5404477\AppData\Local\Temp\ipykernel_4132\693871328.py:32: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv',
C:\Users\z5404477\AppData\Local\Temp\ipykernel_4132\693871328.py:32: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv',
C:\Users\z5404477\AppData\Local\Temp\ipykernel_4132\693871

         station_number             datetime  \
1747282           66037  2022-12-01 00:00:00   
1747283           66037  2022-12-01 00:30:00   
1747284           66037  2022-12-01 01:00:00   
1747285           66037  2022-12-01 01:30:00   
1747286           66037  2022-12-01 02:00:00   

         cloud_amount_of_first_group_in_eighths  \
1747282                                     NaN   
1747283                                     NaN   
1747284                                     NaN   
1747285                                     NaN   
1747286                                     2.0   

        quality_of_first_group_of_cloud_amount  \
1747282                                    NaN   
1747283                                    NaN   
1747284                                    NaN   
1747285                                    NaN   
1747286                                      N   

         cloud_amountof_second_group_in_eighths  \
1747282                                     NaN   
1

In [7]:
filtered_df

,station_number,datetime,cloud_amount_of_first_group_in_eighths,quality_of_first_group_of_cloud_amount,cloud_amountof_second_group_in_eighths,quality_of_second_group_of_cloud_amount,cloud_amount_of_third_group_in_eighths,quality_of_third_group_of_cloud_amount,cloud_amount_of_fourth_group_in_eighths,quality_of_fourth_group_of_cloud_amount,ceilometer_cloud_amount_of_first_group,quality_of_first_group_of_ceilometer_cloud_amount,ceilometer_cloud_amount_of_second_group,quality_of_second_group_of_ceilometer_cloud_amount,ceilometer_cloud_amount_of_third_group,quality_of_third_group_of_ceilometer_cloud_amount,ceilometer_sky_clear_flag
1747282,66037,2022-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OVC,N,NaN,NaN,NaN,NaN,0.0
1747283,66037,2022-12-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OVC,N,NaN,NaN,NaN,NaN,0.0
1747284,66037,2022-12-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OVC,N,NaN,NaN,NaN,NaN,0.0
1747285,66037,2022-12-01 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OVC,N,NaN,NaN,NaN,NaN,0.0
1747286,66037,2022-12-01 02:00:00,2.0,N,7.0,N,NaN,NaN,NaN,NaN,OVC,N,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25016149,66037,2022-05-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25016150,66037,2022-05-31 22:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25016151,66037,2022-05-31 22:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25016152,66037,2022-05-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [8]:
filtered_df['cloud_amount_of_first_group_in_eighths'].value_counts()

cloud_amount_of_first_group_in_eighths
1.0    103267
2.0     28556
3.0     20335
4.0      6281
5.0      4698
6.0      1871
7.0      1860
8.0       453
Name: count, dtype: int64

In [9]:
filtered_df['cloud_amount_of_first_group_in_eighths'].value_counts().sum()

np.int64(167321)

In [95]:
filtered_df['ceilometer_sky_clear_flag'].value_counts()

ceilometer_sky_clear_flag
1.0    142848
0.0    142499
Name: count, dtype: int64

In [10]:
filtered_df['ceilometer_sky_clear_flag'].value_counts()


ceilometer_sky_clear_flag
1.0    142848
0.0    142499
Name: count, dtype: int64